# Repeatable Statistical Inference Demo - Statistical Query

Standard Monte Carlo and Importance Sampling with different sample sizes

In [ ]:
# Add parent directory to path for imports
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# Import our implementations
from distributions import (
    Distribution01, BernoulliRare, BetaSkewed, 
    BimodalMixture, UniformSpike, TruncatedNormal
)

# Set up plotting style with larger fonts
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 14          # Increased from 12
plt.rcParams['axes.labelsize'] = 16     # Axis labels
plt.rcParams['axes.titlesize'] = 18     # Title size
plt.rcParams['xtick.labelsize'] = 14    # X-axis tick labels
plt.rcParams['ytick.labelsize'] = 14    # Y-axis tick labels
plt.rcParams['legend.fontsize'] = 14    # Legend font size
plt.rcParams['figure.titlesize'] = 20   # Figure title size

print("All imports successful!")

All imports successful!


In [ ]:
# Create all 6 distributions for comprehensive testing
print("Creating distributions...")
distributions = [
    BernoulliRare(p=0.05),                     # 1. Rare events
    BetaSkewed(alpha=0.5, beta=2.0),           # 2. Right-skewed
    TruncatedNormal(mu=0.3, sigma=0.15),       # 3. Truncated normal
    BimodalMixture(weight=0.3)                # 4. Bimodal mixture
]

Creating distributions...


In [ ]:
def apply_truncated_importance_sampling(samples, weights, truncation_threshold=50.0):
    """Apply truncated importance sampling with bias correction"""
    # Truncate weights
    truncated_weights = np.minimum(weights, truncation_threshold)
    
    # Compute bias correction factor
    n_truncated = np.sum(weights > truncation_threshold)
    if n_truncated > 0:
        # Simple bias correction: adjust the normalization
        total_raw_weight = np.sum(weights)
        total_truncated_weight = np.sum(truncated_weights)
        
        if total_truncated_weight > 0:
            # Bias correction factor to approximately restore the mean
            bias_correction = total_raw_weight / total_truncated_weight
            # Apply correction but cap it to prevent overcorrection
            bias_correction = min(bias_correction, 2.0)  # Cap correction at 2x
            truncated_weights *= bias_correction
    
    return truncated_weights

def importance_sampling(target_dist: Distribution01, importance_dist: Distribution01, 
                                n_samples: int, seed: int = None) -> dict:
    """Enhanced importance sampling with proper truncated IS"""
    if seed is not None:
        np.random.seed(seed)
    
    # Sample from importance distribution
    samples = importance_dist.sample(n_samples)
    
    # Compute importance weights
    target_pdf = target_dist.pdf(samples)
    importance_pdf = importance_dist.pdf(samples)
    
    # Avoid division by zero
    weights = np.where(importance_pdf > 1e-12, target_pdf / importance_pdf, 0.0)
    
    # Apply proper truncated importance sampling with bias correction
    weights = apply_truncated_importance_sampling(samples, weights, truncation_threshold=50.0)
    
    # Weighted average with proper normalization
    if np.sum(weights) > 0:
        normalized_weights = weights / np.sum(weights)
        estimate = np.sum(samples * weights) / np.sum(weights)
        
        # Compute effective sample size
        eff_sample_size = 1.0 / np.sum(normalized_weights**2)
        
        # Variance estimation for importance sampling
        weighted_variance = np.sum(normalized_weights * (samples - estimate)**2)
        standard_error = np.sqrt(weighted_variance / eff_sample_size)
    else:
        estimate = 0.0
        eff_sample_size = 0.0
        standard_error = float('inf')
        normalized_weights = weights
    
    return {
        'estimate': estimate,
        'standard_error': standard_error,
        'effective_sample_size': eff_sample_size,
        'efficiency': eff_sample_size / n_samples if n_samples > 0 else 0.0,
        'weight_stats': {
            'min': np.min(weights),
            'max': np.max(weights),
            'mean': np.mean(weights),
            'std': np.std(weights),
            'cv': np.std(weights) / np.mean(weights) if np.mean(weights) > 0 else float('inf')
        }
    }

def monte_carlo_sampling(distribution: Distribution01, n_samples: int, seed: int = None) -> dict:
    """Enhanced Monte Carlo with confidence intervals"""
    if seed is not None:
        np.random.seed(seed)
    samples = distribution.sample(n_samples)
    estimate = np.mean(samples)
    standard_error = np.std(samples, ddof=1) / np.sqrt(n_samples)
    
    return {
        'estimate': estimate,
        'standard_error': standard_error,
        'effective_sample_size': n_samples,
        'efficiency': 1.0,
        'weight_stats': {
            'min': 1.0, 'max': 1.0, 'mean': 1.0, 'std': 0.0, 'cv': 0.0
        }
    }

In [ ]:
# Comprehensive comparison across sample sizes and distributions
sample_sizes = [200, 500, 1000, 2000, 5000, 10000]
results_comprehensive = {}

print("=== MONTE CARLO vs IMPORTANCE SAMPLING COMPARISON ===")
print(f"Testing {len(distributions)} distributions across {len(sample_sizes)} sample sizes")
print("=" * 80)

for dist in distributions:
    print(f"\n📊 DISTRIBUTION: {dist.name}")
    print(f"   True mean: {dist.true_mean():.6f}")
    
    importance_dist = dist.suggest_importance_distribution()
    use_importance = importance_dist.name != dist.name
    
    if use_importance:
        print(f"   Importance distribution: {importance_dist.name}")
    else:
        print("   No importance sampling benefit expected")
    
    dist_results = {
        'monte_carlo': {},
        'importance_sampling': {} if use_importance else None,
        'true_mean': dist.true_mean()
    }
    
    # Test each sample size
    print(f"\n   {'n':<8} {'MC Est':<10} {'MC SE':<8} {'IS Est':<10} {'IS SE':<8} {'Eff Gain':<8} {'IS Eff%':<8}")
    print("   " + "-" * 70)
    
    for n in sample_sizes:
        # Monte Carlo
        mc_result = monte_carlo_sampling(dist, n, seed=42)
        mc_error = abs(mc_result['estimate'] - dist.true_mean())
        dist_results['monte_carlo'][n] = {
            'estimate': mc_result['estimate'],
            'standard_error': mc_result['standard_error'],
            'error': mc_error,
            'relative_error': mc_error / dist.true_mean() if dist.true_mean() != 0 else mc_error
        }
        
        # Importance Sampling
        if use_importance:
            is_result = importance_sampling(dist, importance_dist, n, seed=42)
            is_error = abs(is_result['estimate'] - dist.true_mean())
            dist_results['importance_sampling'][n] = {
                'estimate': is_result['estimate'],
                'standard_error': is_result['standard_error'],
                'error': is_error,
                'relative_error': is_error / dist.true_mean() if dist.true_mean() != 0 else is_error,
                'efficiency': is_result['efficiency'],
                'effective_sample_size': is_result['effective_sample_size']
            }
            
            # Efficiency metrics
            variance_ratio = (mc_result['standard_error']**2) / (is_result['standard_error']**2) if is_result['standard_error'] > 0 else float('inf')
            
            print(f"   {n:<8} {mc_result['estimate']:<10.4f} {mc_result['standard_error']:<8.4f} "
                  f"{is_result['estimate']:<10.4f} {is_result['standard_error']:<8.4f} "
                  f"{variance_ratio:<8.2f} {100*is_result['efficiency']:<8.1f}")
        else:
            print(f"   {n:<8} {mc_result['estimate']:<10.4f} {mc_result['standard_error']:<8.4f} "
                  f"{'N/A':<10} {'N/A':<8} {'N/A':<8} {'N/A':<8}")
    
    results_comprehensive[dist.name] = dist_results
    print()

=== MONTE CARLO vs IMPORTANCE SAMPLING COMPARISON ===
Testing 6 distributions across 6 sample sizes

📊 DISTRIBUTION: Bernoulli(0.050)
   True mean: 0.050000
   Importance distribution: Bernoulli(0.150)

   n        MC Est     MC SE    IS Est     IS SE    Eff Gain IS Eff% 
   ----------------------------------------------------------------------
   200      0.0400     0.0139   0.0519     0.0163   0.73     92.5    
   500      0.0600     0.0106   0.0560     0.0107   0.98     92.0    
   1000     0.0460     0.0066   0.0511     0.0072   0.84     92.6    
   2000     0.0525     0.0050   0.0509     0.0051   0.95     92.6    
   5000     0.0512     0.0031   0.0485     0.0032   0.98     92.9    
   10000    0.0474     0.0021   0.0481     0.0022   0.92     93.0    


📊 DISTRIBUTION: Beta(0.5, 2.0)
   True mean: 0.200000
   No importance sampling benefit expected

   n        MC Est     MC SE    IS Est     IS SE    Eff Gain IS Eff% 
   ------------------------------------------------------------